In [1]:
from sagemaker import Session, LocalSession
import boto3
import sagemaker
import os
from dotenv import load_dotenv
load_dotenv()

config_env = 'default'
boto_session = boto3.Session(profile_name=config_env)
iam = boto_session.client('iam')
role = iam.get_role(RoleName=os.environ["SAGEMAKER_EXECUTION_ROLE"])['Role']['Arn']

sagemaker_session = Session(boto_session=boto_session)

# LocalSession
# sagemaker_session = LocalSession(boto_session=boto_session)
# sagemaker_session.config = {'local': {'local_code': True}}

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/steve/.config/sagemaker/config.yaml


In [2]:
import os

# zip the local model folder to model.tar.gz - only the contents of the folder, not the folder itself
!cd model && tar -czvf ../model.tar.gz *

# # upload to s3 if it doesn't exist
# # Create an S3 client

s3 = boto3.client('s3')
#                              This is the default sagemaker bucket - but can probably grab a different bucket with the same permissions
s3.upload_file('model.tar.gz', os.environ["SAGEMAKER_S3_BUCKET"], 'uv_net.tar.gz')


best.bak


best.ckpt
code/
code/inference.py
code/demo_input.pkl
code/encoders.py
code/test_inference.py
code/requirements.txt
code/models.py
code/util.py
hparams.yaml


In [3]:

from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(
    model_data="./model.tar.gz",
    entry_point="inference.py",
    framework_version="1.13",
    py_version="py39",
    # source_dir="code",
    role=role,
    sagemaker_session=sagemaker_session,
)

In [9]:
# create dummy payload
import pickle

with open("demo_input.pkl", "rb") as f:
    payload = pickle.load(f)

payload['graph_face_feat'] = payload['graph_face_feat'].tolist()
payload['graph_edge_feat'] = payload['graph_edge_feat'].tolist()

In [18]:

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


--------!

In [ ]:
# SERVERLESS CONFIG
# TODO: test to see configuration that actually works
# deploying from local breaks
# and deploying from remote timesout with current config
# from sagemaker.serverless import ServerlessInferenceConfig

# Create an empty ServerlessInferenceConfig object to use default values
# serverless_config = ServerlessInferenceConfig(
#     memory_size_in_mb=2048,
#     provisioned_concurrency=1,
# )

# serverless_predictor = model.deploy(
#     endpoint_name="cad-feature-detection-uv-net",
#     serverless_inference_config=serverless_config,
#     serializer=JSONSerializer(),
#     deserializer=JSONDeserializer(),
# )

In [ ]:
# Local or On Demand
predictor = model.deploy(
    initial_instance_count=1,  
    instance_type="ml.t2.medium", # local
    endpoint_name="cad-feature-detection-uv-net",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [19]:
response = predictor.predict(payload)

In [20]:
response

['triangular_passage',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'chamfer',
 'triangular_blind_step',
 'chamfer',
 'chamfer',
 'triangular_blind_step',
 'chamfer',
 'chamfer',
 'chamfer',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'chamfer',
 'chamfer',
 'chamfer',
 'chamfer',
 'chamfer',
 'chamfer',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_blind_step',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 'triangular_passage',
 '

In [ ]:
type(response)

list

In [22]:
# Clean Up
predictor.delete_model()
predictor.delete_endpoint()

# serverless_predictor.delete_model()
# serverless_predictor.delete_endpoint()
